In [ ]:
import arcpy
from arcpy.sa import *
import math

# Ensure the Spatial Analyst extension is available
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
else:
    raise RuntimeError("Spatial Analyst extension is not available.")

# Set file paths (manually specify paths here)
thermal_band_path = r"C:\path\to\your\workspace\thermal_band.tif"  # Path to Landsat 8 thermal band (Band 10)
study_area_path = r"C:\path\to\your\workspace\study_area.shp"      # Path to shapefile or feature class of study area
clipped_thermal_path = r"C:\path\to\your\workspace\clipped_thermal.tif"  # Output path for clipped thermal raster
toa_radiance_path = r"C:\path\to\your\workspace\toa_radiance.tif"        # Output path for TOA Radiance raster
brightness_temperature_path = r"C:\path\to\your\workspace\brightness_temp.tif"  # Output path for Brightness Temperature raster
output_lst_path = r"C:\path\to\your\workspace\lst_output.tif"  # Final output path for LST raster

# Metadata constants (adjust values as per Landsat metadata)
ML = 0.0003342  # Radiance multiplicative scaling factor
AL = 0.1        # Radiance additive scaling factor
K1 = 774.8853   # K1 constant (W/m^2/sr/μm)
K2 = 1321.0789  # K2 constant (Kelvin)
wavelength = 10.895  # Central wavelength of Band 10 in micrometers
rho = 14380.0  # Constant for Planck's law
emissivity = 0.96  # Surface emissivity (adjust if needed)

# Set environment settings
arcpy.env.overwriteOutput = True  # Allow overwriting of output files

try:
    # Step 1: Clip the thermal band to the study area
    print("Clipping the thermal band to the study area...")
    arcpy.management.Clip(
        in_raster=thermal_band_path,
        rectangle="#",
        out_raster=clipped_thermal_path,
        in_template_dataset=study_area_path,
        nodata_value="#",
        clipping_geometry="ClippingGeometry",
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )

    # Step 2: Convert DN to Top of Atmosphere (TOA) Radiance
    print("Calculating TOA Radiance...")
    toa_radiance = Raster(clipped_thermal_path) * ML + AL
    toa_radiance.save(toa_radiance_path)  # Save TOA Radiance raster

    # Step 3: Convert TOA Radiance to Brightness Temperature (Kelvin)
    print("Calculating Brightness Temperature...")
    brightness_temperature = (K2 / Ln((K1 / Raster(toa_radiance_path)) + 1)) - 273.15
    brightness_temperature.save(brightness_temperature_path)  # Save Brightness Temperature raster

    # Step 4: Calculate LST using Brightness Temperature and emissivity
    print("Calculating Land Surface Temperature (LST)...")
    lst = brightness_temperature / (1 + (wavelength * brightness_temperature / rho) * math.log(emissivity))
    lst.save(output_lst_path)  # Save the final LST raster

    print(f"LST calculation completed successfully. Output saved at: {output_lst_path}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Release the Spatial Analyst extension
    arcpy.CheckInExtension("Spatial")
